# Neural Network for Classification from Scratch

---

Author: JGStrickland

Date: July 2025

Building a Neural Network for Classification from Scratch
This notebook demonstrates how to build a neural network for classification tasks using Python and NumPy, without relying on high-level machine learning libraries. It covers fundamental concepts and implementation details of neural networks for classification problems.

Steps Covered:

- Network Architecture – Designing a feedforward neural network with configurable layers, neurons, and activation functions
- Initialization – Implementing proper weight initialization techniques (e.g., Xavier/Glorot initialization)
- Forward Propagation – Computing predictions through the network using activation functions (sigmoid, ReLU, softmax)
- Loss Calculation – Implementing cross-entropy loss for classification tasks
- Backpropagation – Calculating gradients using the chain rule efficiently
- Parameter Updates – Applying optimization algorithms (e.g., gradient descent, Adam)
- Training Loop – Implementing iterative training with batch processing
- Prediction – Making class predictions on new data
- Evaluation – Measuring model performance using accuracy, precision, recall, and F1-score
- Testing – Validating the model on real-world classification datasets

This notebook provides a hands-on introduction to the fundamentals of neural networks and demonstrates how the underlying mathematics of deep learning can be implemented programmatically for classification tasks.

The implementation will include a flexible neural network class that can be configured with different architectures, making it suitable for various classification problems including binary and multi-class classification.

## Step Zero: Imports

---

We will use Numpy to handle the matrix multiplication needed in this project. Numpy is preferred over native Python lists because it is much faster, more memory-efficient, and provides built-in support for linear algebra operations like matrix multiplication, transposes, and inverses.



In [47]:
import numpy as np

## Step One: Designing the Architecture

---

Here we will begin by creating the relevant classes for our classifier, starting off with a Layer class

### Layering up

---

A neural network consists of multiple layers of interconnected neurons, where each layer transforms its input through a weighted sum, bias addition, and non-linear activation function. Let's examine the Layer class attributes in detail:

- weights: Matrix (output_size × input_size) that determines the strength of connections between neurons in adjacent layers. Proper initialization is crucial - we use Xavier/Glorot initialization to maintain stable gradients during training.
- biases: Vector (output_size × 1) added to the weighted sum, allowing the network to learn offsets and improve model flexibility. Initialized to zeros to start from a symmetric state.
- activation: String specifying the non-linear function applied to the layer's output. Hidden layers typically use ReLU, while the output layer uses softmax for classification probabilities.
- input: Stores the input values received by this layer during forward propagation, necessary for computing gradients during backpropagation.
- output: Stores the activated output values after applying the activation function, serving as input to the next layer.
- z: Stores the intermediate linear transformation (weighted sum plus bias) before applying the activation function, needed for gradient calculations.

We will explore each of these attributes later in this notebook.

### Connecting the Layers

---

The NeuralNetworkClassifier class orchestrates the complete network architecture by connecting multiple layers in sequence. Let's examine its attributes:

- layers: Ordered list containing all Layer objects that constitute the neural network, typically including input, hidden, and output layers.
- input_size: Integer specifying the number of input features to the network.
- output_size: Integer specifying the number of output classes for classification.
- parameters: Dictionary that consolidates all trainable parameters (weights and biases) from all layers, facilitating efficient parameter updates during optimization.
- gradients: Dictionary that stores computed gradients for each parameter during backpropagation, used to update weights and biases during training.
- prev_size: Tracks the output dimension of the most recently added layer, ensuring dimensional compatibility when adding subsequent layers. Initialized to the input_size.

We will explore each of these attributes later in this notebook.

In [48]:
class Layer:
    def __init__(self, activation):
        self.weights = np.array([])
        self.biases = np.array([])
        self.activation = activation
        self.input = None
        self.output = None
        self.z = None


class NeuralNetworkClassifier:
    def __init__(self, input_size, activation, hidden_layers=[64, 32], output_size=2):
        total_layers = len(hidden_layers) + 1  # hidden layers + output layer
        self.layers = np.empty(total_layers, dtype=object)
        self.input_size = input_size
        self.output_size = output_size
        self.parameters = {}
        self.gradients = {}

        # Track dimensions for layer connections
        prev_size = input_size

### Creating the neurons

---

Now we have initialised the classes we still need to add some functionality before we move on to the next step. Here we are going to expand on the NeuralNetworkClassifier class to create the layers when the class is initalised. This will leave us with ...

In [49]:
class NeuralNetworkClassifier:
    def __init__(self, input_size, activation, hidden_layers=[64, 32], output_size=2):
        total_layers = len(hidden_layers) + 1  # hidden layers + output layer
        self.layers = np.empty(total_layers, dtype=object)
        self.input_size = input_size
        self.output_size = output_size
        self.parameters = {}
        self.gradients = {}

        # Track dimensions for layer connections
        prev_size = input_size

        # Add hidden layers
        for i, layer_size in enumerate(hidden_layers):
            layer = Layer(prev_size, layer_size, activation)
            self.layers = np.append(self.layers, layer)
            self.parameters[f'W{i}'] = layer.weights
            self.parameters[f'b{i}'] = layer.biases
            prev_size = layer_size

        # Add output layer (always uses softmax for classification)
        output_layer = Layer(prev_size, output_size, 'softmax')
        self.layers = np.append(self.layers, output_layer)
        self.parameters[f'W{len(hidden_layers)}'] = output_layer.weights
        self.parameters[f'b{len(hidden_layers)}'] = output_layer.biases

## Step Two: Initialising the weights

---

### Xavier/Glorot initialization

Xavier/Glorot initialization is a technique for initializing the weights of neural networks that helps to stabilize and accelerate training, especially in deep networks. When weights are initialized with values that are too large or too small, it can lead to:

- Vanishing gradients: Gradients become extremely small, slowing or stopping learning
- Exploding gradients: Gradients become extremely large, causing unstable training
- Saturated activation functions: Neurons get "stuck" in flat regions of activation functions

Xavier initialization sets the initial weights by drawing them from a distribution with:

Zero mean
Variance specifically calculated based on the number of input and output neurons
The formula for the variance is:

$$
Variance = \frac{2}{\left(n_{input} + n_{output}\right)}
$$

where:

- $n_{input}$ = the number of neurons in the previous layer
- $n_{output}$ = the number of neurons in the current layer

In this step we will also initialise the biases to zeros ....



In [50]:
class Layer:
    def __init__(self, input_size, output_size, activation):
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.biases = np.zeros((output_size, 1))
        self.activation = activation
        self.input = None
        self.output = None
        self.z = None

## Step Three: Forward Propagation

---

The forward propagation process involves two main mathematical operations at each layer:

### Linear Transformation

---

$$
z=W⋅x+b
$$

Where:
- $W$ is the weight matrix of size (output_size × input_size)
- $x$ is the input vector of size (input_size × 1)
- $b$ is the bias vector of size (output_size × 1)
- $z$ is the intermediate result before activation


In [51]:
class Layer:
    def __init__(self, input_size, output_size, activation):
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.biases = np.zeros((output_size, 1))
        self.activation = activation
        self.input = None
        self.output = None
        self.z = None


    def forward(self, input):
        self.input = input
        self.z = np.dot(self.weights, self.input) + self.biases
        self.output = self.z
        return self.output

### Non-linear Activation:

---

$$
a=f(z)
$$

Where $f$ is the activation function (ReLU, sigmoid, or softmax)

When processing a batch of inputs, the input matrix $X$ has shape (input_size × batch_size), and the matrix multiplication $W \cdot X$ results in a matrix of size (output_size × batch_size). The bias $b$ is added to each column of this result through broadcasting.

The activation functions introduce non-linearity:

- ReLU: $f(z) = \max(0, z)$ - Helps with vanishing gradient problem
- Sigmoid: $f(z) = \frac{1}{1 + e^{-z}}$ - Maps values to (0, 1) range
- Softmax: $f(z_i) = \frac{e^{z_i}}{\sum_j e^{z_j}}$ - Converts logits to probabilities

The softmax function includes a numerical stability trick: subtracting the maximum value before exponentiation to prevent overflow while maintaining the same relative probabilities.

This forward propagation process transforms the input through each layer sequentially, with the output of one layer serving as the input to the next, ultimately producing class probabilities at the final layer.

In [52]:
class Layer:
    def __init__(self, input_size, output_size, activation):
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.biases = np.zeros((output_size, 1))
        self.activation = activation
        self.input = None
        self.output = None
        self.z = None


    def forward(self, input):
        self.input = input
        self.z = np.dot(self.weights, self.input) + self.biases
        self.output = self.apply_activation(self.z)
        return self.output


    def apply_activation(self, z):
        if self.activation == 'relu':
            return np.maximum(0, z)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-z))
        elif self.activation == 'softmax':
            exp_z = np.exp(z - np.max(z))  # Numerical stability
            return exp_z / np.sum(exp_z, axis=0)
        else:
            raise ValueError("Unsupported activation function")


class NeuralNetworkClassifier:
    def __init__(self, input_size, activation, hidden_layers=[64, 32], output_size=2):
        total_layers = len(hidden_layers) + 1  # hidden layers + output layer
        self.layers = np.empty(total_layers, dtype=object)
        self.input_size = input_size
        self.output_size = output_size
        self.parameters = {}
        self.gradients = {}

        # Track dimensions for layer connections
        prev_size = input_size

        # Add hidden layers
        for i, layer_size in enumerate(hidden_layers):
            layer = Layer(prev_size, layer_size, activation)
            self.layers = np.append(self.layers, layer)
            self.parameters[f'W{i}'] = layer.weights
            self.parameters[f'b{i}'] = layer.biases
            prev_size = layer_size

        # Add output layer (always uses softmax for classification)
        output_layer = Layer(prev_size, output_size, 'softmax')
        self.layers = np.append(self.layers, output_layer)
        self.parameters[f'W{len(hidden_layers)}'] = output_layer.weights
        self.parameters[f'b{len(hidden_layers)}'] = output_layer.biases


    def forward(self, X):
        # Propagate input through all layers.
        input = X
        for layer in self.layers:
            input = layer.forward(input)
        return input

## Step Four: Loss Calculation

----

Loss calculation is a critical step in training neural networks as it measures how well the model's predictions match the true labels. For classification problems, we use Cross-Entropy Loss (also known as log loss), which quantifies the difference between predicted probabilities and actual class distributions.

### Cross-Entropy Loss

----

The cross-entropy loss for a single sample is defined as:

$$
J = -\sum^{C}_{i=1}y_{i}\log\left(\hat{y}_{i}\right)
$$

Where:

- $C$ = number of classes
- $y_i$ = true label (1 for correct class, 0 for others - one-hot encoded)
- $\hat{y}_i$ = predicted probability for class $i$

For a batch of $N$ samples, the average loss is:

$$
J = - \frac{1}{N}\sum_{n=1}^{N}\sum_{i=1}^{C}y_{i}^{\left(n\right)}\log\left(\hat{y}_{i}^{\left(n\right)}\right)
$$

### Implementation Considerations

---

- Numerical stability: We add a small epsilon to prevent log(0) which would be undefined
- Batch processing: We compute the average loss across all samples in the batch
- One-hot encoding: True labels must be converted to one-hot format for multi-class classification




In [53]:
class NeuralNetworkClassifier:
    def __init__(self, input_size, activation, hidden_layers=[64, 32], output_size=2):
        total_layers = len(hidden_layers) + 1  # hidden layers + output layer
        self.layers = np.empty(total_layers, dtype=object)
        self.input_size = input_size
        self.output_size = output_size
        self.parameters = {}
        self.gradients = {}

        # Track dimensions for layer connections
        prev_size = input_size

        # Add hidden layers
        for i, layer_size in enumerate(hidden_layers):
            layer = Layer(prev_size, layer_size, activation)
            self.layers = np.append(self.layers, layer)
            self.parameters[f'W{i}'] = layer.weights
            self.parameters[f'b{i}'] = layer.biases
            prev_size = layer_size

        # Add output layer (always uses softmax for classification)
        output_layer = Layer(prev_size, output_size, 'softmax')
        self.layers = np.append(self.layers, output_layer)
        self.parameters[f'W{len(hidden_layers)}'] = output_layer.weights
        self.parameters[f'b{len(hidden_layers)}'] = output_layer.biases


    def forward(self, X):
        # Propagate input through all layers.
        input = X
        for layer in self.layers:
            input = layer.forward(input)
        return input


    def compute_loss(self, y_pred, y_true, epsilon=1e-12):
        # Ensure numerical stability by clipping predictions
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)

        # Compute cross-entropy loss for each sample
        # -y_true * log(y_pred) and sum across classes
        sample_losses = -np.sum(y_true * np.log(y_pred), axis=0)

        # Return average loss J across the batch
        J = np.mean(sample_losses)
        return J


## Step Five: Backpropagation

---

Backpropagation is the algorithm that allows neural networks to learn from data by calculating how each weight and bias should be adjusted to reduce the loss. It works by propagating the error backwards through the network, using the chain rule from calculus to compute gradients efficiently.

To fully understand back propagation we must first understand the chain rule in regards to derivatives and partial derivatives.

### Derivatives

---

Finding the derivative of a function allows yyou to understand how much a function changes when as the variables do.

So given the function $f\left(x\right)$, the derivative $f'\left(x\right)$ tells us how $f\left(x\right)$ changes as $x$ changes.

### Partial Derivatives

---

Partial derivatives measure how a function changes when only one of its input variables changes, while keeping all other variables constant. In neural networks, we use partial derivatives to understand how the loss function $J$ changes with respect to each individual parameter (weights and biases).
Mathematically, the partial derivative of $J$ with respect to a weight $w$ is denoted as:

$$
\frac{\partial J}{\partial w}
$$

This tells us the "direction and rate" at which $J$ changes as we make small adjustments to $w$.

- If $\frac{\partial J}{\partial w} > 0$, increasing $w$ increases the loss, so we should reduce $w$.
- If $\frac{\partial J}{\partial w} < 0$, increasing $w$ decreases the loss, so we should increase $w$.

### The Chain Rule

---

The chain rule is a fundamental calculus principle that allows us to compute derivatives of composite functions. If a variable $z$ depends on $y$, and $y$ depends on $x$, then:

$$
\frac{dz}{dx} = \frac{dz}{dy} . \frac{dy}{dx}
$$

### Applying the chain rule to neural networks

---

In a neural network, we have the following sequence of computations:

1. Linear step:

$$
z^{l} = W^{l}a^{l-1}+b^{l}
$$

2. Activation step:

$$
a^{l} = f\left(z^{l}\right)
$$

3. Output step:

$$
\hat{y} = a^{L}
$$

4. Loss function:

$$
J = Loss\left(\hat{y},y\right)
$$

So the dependencies look like:

$$
J \to \hat{y} \to a \to z \to W,b
$$

By the chain rule, we can express the gradient of the loss with respect to he weights $W$ as:

$$
\frac{\partial J}{\partial W} = \frac{\partial J}{\partial \hat{y}} . \frac{\partial \hat{y}}{\partial a} . \frac{\partial a}{\partial z} . \frac{\partial z}{\partial W}
$$

The bias chain is shorter because the derivative $\frac{\partial z}{\partial b}$ is always 1, while $\frac{\partial z}{\partial W}$ depends on the inputs and requires carrying more terms from the chain. So can be shortened down too simply:

$$
\frac{\partial J}{\partial b} = \frac{\partial J}{\partial z}
$$

This then tells us exactly how much each weight and bias contributed to the error, and therefore how they should be updated in order to reduce the loss.

### Error signals

---

To make backpropagation efficient, instead of repeatedly applying the chain rule directly to every weight and bias, we define the error term ($\delta$) for each layer:

$$
\delta^{l} = \frac{\partial J}{\partial z^{l}}
$$

Here:
- $z^{l}$ is the weighted input to layer $l$ (before applying the activation function).
- $\delta^{l}$ measures how much the loss would change if we changed the pre-activation of this layer.
- Intuitively, it tells us how “wrong” the layer’s outputs are and how strongly they need to be corrected.

Instead of computing gradients separately for each weight and bias, we compute $\delta$ once per neuron per layer. Once $\delta$ is known, the gradients for weights and biases follows:

- Weight gradients:

$$
\frac{\partial J}{\partial W^{l}} = \delta^{l}\left(a^{l-1}\right)^{T}
$$

This multiplies the error by the activations from the previous layer, telling us how each weight contributed to the error.

- Bias gradients:

$$
\frac{\partial J}{\partial b^{l}} = \delta^{l}
$$

Since the bias is added directly to $z^{l}$, its gradient is just the error itself.

To send the error backward to the previous layer we compute:

$$
\delta^{l-1} = \left(W^{l}\right)^{T}\delta^{l}\odot f'\left(z^{z-1}\right)
$$

- $\left(W^{l}\right)^{T}\delta^{l}$ propagates the next layer's error backward through the weights.
- $\odot f'\left(z^{z-1}\right)$ multiplies this error element-wise by the derivative of the activation function for each neuron in the previous layer.
- This combination tells us how much each neuron in layer $l-1$ contributed to the error in layer $l$, allowing the network to update all weights and biases efficiently.

### Activation Function Derivatives

---

When applying the chain rule, an important term is:

$$
\frac{\partial a}{\partial z}
$$

This is the derivative of the activation function, which determines how the gradient flows backward through the network. Different activation functions have different derivatives:

- **Sigmoid**:
  If $a = \sigma(z) = \frac{1}{1+e^{-z}}$, then
  $$
  \frac{\partial a}{\partial z} = a(1-a).
  $$

- **ReLU (Rectified Linear Unit)**:
  If $a = \max(0, z)$, then
  $$
  \frac{\partial a}{\partial z} =
  \begin{cases}
  1 & z > 0 \\
  0 & z \leq 0
  \end{cases}
  $$

- **Softmax** (with cross-entropy loss):
  If $a_i = \frac{e^{z_i}}{\sum_j e^{z_j}}$, then
  $$
  \frac{\partial a_i}{\partial z_j} = a_i (\delta_{ij} - a_j),
  $$
  where $\delta_{ij}$ is the Kronecker delta.
  In practice, when combined with cross-entropy loss, this simplifies to:
  $$
  \frac{\partial J}{\partial z_i} = a_i - y_i.
  $$

These derivatives are plugged into the chain rule formulas for $\frac{\partial J}{\partial W}$ and $\frac{\partial J}{\partial b}$, allowing the error signal to propagate backward correctly.





In [54]:
class NeuralNetworkClassifier:
    def __init__(self, input_size, activation, hidden_layers=[64, 32], output_size=2):
        total_layers = len(hidden_layers) + 1  # hidden layers + output layer
        self.layers = np.empty(total_layers, dtype=object)
        self.input_size = input_size
        self.output_size = output_size
        self.parameters = {}
        self.gradients = {}

        # Track dimensions for layer connections
        prev_size = input_size

        # Add hidden layers
        for i, layer_size in enumerate(hidden_layers):
            layer = Layer(prev_size, layer_size, activation)
            self.layers = np.append(self.layers, layer)
            self.parameters[f'W{i}'] = layer.weights
            self.parameters[f'b{i}'] = layer.biases
            prev_size = layer_size

        # Add output layer (always uses softmax for classification)
        output_layer = Layer(prev_size, output_size, 'softmax')
        self.layers = np.append(self.layers, output_layer)
        self.parameters[f'W{len(hidden_layers)}'] = output_layer.weights
        self.parameters[f'b{len(hidden_layers)}'] = output_layer.biases


    def forward(self, X):
        # Propagate input through all layers.
        input = X
        for layer in self.layers:
            input = layer.forward(input)
        return input


    def compute_loss(self, y_pred, y_true, epsilon=1e-12):
        # Ensure numerical stability by clipping predictions
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)

        # Compute cross-entropy loss for each sample
        # -y_true * log(y_pred) and sum across classes
        sample_losses = -np.sum(y_true * np.log(y_pred), axis=0)

        # Return average loss J across the batch
        J = np.mean(sample_losses)
        return J


    def backward(self, X, y_true, y_pred):
        batch_size = y_true.shape[1]

        # Start with output layer gradient (softmax cross-entropy derivative)
        dZ = (y_pred - y_true) / batch_size  # Average gradient over batch

        # Backpropagate through layers
        for l in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[l]

            # Compute gradients for current layer
            self.gradients[f'dW{l}'] = np.dot(dZ, layer.input.T)
            self.gradients[f'db{l}'] = np.sum(dZ, axis=1, keepdims=True)

            if l > 0:  # Compute gradient for previous layer
                dA_prev = np.dot(layer.weights.T, dZ)
                dZ = dA_prev * self.layers[l - 1].activation_derivative(self.layers[l - 1].z)


class Layer:
    def __init__(self, input_size, output_size, activation):
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.biases = np.zeros((output_size, 1))
        self.activation = activation
        self.input = None
        self.output = None
        self.z = None


    def forward(self, input):
        self.input = input
        self.z = np.dot(self.weights, self.input) + self.biases
        self.output = self.apply_activation(self.z)
        return self.output


    def apply_activation(self, z):
        if self.activation == 'relu':
            return np.maximum(0, z)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-z))
        elif self.activation == 'softmax':
            exp_z = np.exp(z - np.max(z))  # Numerical stability
            return exp_z / np.sum(exp_z, axis=0)
        else:
            raise ValueError("Unsupported activation function")


    def activation_derivative(self, z):
        if self.activation == 'relu':
            return (z > 0).astype(float)
        elif self.activation == 'sigmoid':
            s = 1 / (1 + np.exp(-z))
            return s * (1 - s)
        elif self.activation == 'softmax':
            # Note: Softmax derivative is handled separately in cross-entropy loss
            raise ValueError("Should not use softmax derivative directly in backprop")
        else:
            raise ValueError("Unsupported activation function")

## Step Six: Parameter Updates

---

Parameter updates are the final step in the training process where we adjust the weights and biases based on the gradients computed during backpropagation. This is where the actual learning occurs as we iteratively improve the model's parameters to minimize the loss function.

### Gradient Descent Optimization

---

The most basic optimization algorithm is Gradient Descent, which updates parameters in the direction opposite to the gradient of the loss function:

$$
W = W - \eta\frac{\partial J}{\partial W}
$$

$$
b = b - \eta\frac{\partial J}{\partial b}
$$

Where:

- $\eta$ is the learning rate, a hyperparameter that controls the step size
- $\frac{\partial J}{\partial W}$ and $\frac{\partial J}{\partial b}$ are the gradients computed during backpropagation

The learning rate is crucial:

- Too small: Slow convergence, may get stuck in local minima
- Too large: May overshoot the minimum, causing divergence

In our implementation, we update all parameters using these simple gradient descent rules, iteratively moving toward the minimum of the loss function.

In [55]:
class NeuralNetworkClassifier:
    def __init__(self, input_size, activation, hidden_layers=[64, 32], output_size=2):
        total_layers = len(hidden_layers) + 1  # hidden layers + output layer
        self.layers = np.empty(total_layers, dtype=object)
        self.input_size = input_size
        self.output_size = output_size
        self.parameters = {}
        self.gradients = {}
        self.loss_history = []

        # Track dimensions for layer connections
        prev_size = input_size

        # Add hidden layers
        for i, layer_size in enumerate(hidden_layers):
            layer = Layer(prev_size, layer_size, activation)
            self.layers[i] = layer
            self.parameters[f'W{i}'] = layer.weights
            self.parameters[f'b{i}'] = layer.biases
            prev_size = layer_size

        # Add output layer (always uses softmax for classification)
        output_layer = Layer(prev_size, output_size, 'softmax')
        self.layers[-1] = output_layer
        self.parameters[f'W{len(hidden_layers)}'] = output_layer.weights
        self.parameters[f'b{len(hidden_layers)}'] = output_layer.biases


    def forward(self, X):
        # Propagate input through all layers.
        input = X
        for layer in self.layers:
            input = layer.forward(input)
        return input


    def compute_loss(self, y_pred, y_true, epsilon=1e-12):
        # Ensure numerical stability by clipping predictions
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)

        # Compute cross-entropy loss for each sample
        # -y_true * log(y_pred) and sum across classes
        sample_losses = -np.sum(y_true * np.log(y_pred), axis=0)

        # Return average loss J across the batch
        J = np.mean(sample_losses)
        return J


    def backward(self, X, y_true, y_pred):
        batch_size = y_true.shape[1]

        # Start with output layer gradient (softmax cross-entropy derivative)
        dZ = (y_pred - y_true) / batch_size  # Average gradient over batch

        # Backpropagate through layers
        for l in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[l]

            # Compute gradients for current layer
            self.gradients[f'dW{l}'] = np.dot(dZ, layer.input.T)
            self.gradients[f'db{l}'] = np.sum(dZ, axis=1, keepdims=True)

            if l > 0:  # Compute gradient for previous layer
                dA_prev = np.dot(layer.weights.T, dZ)
                dZ = dA_prev * self.layers[l - 1].activation_derivative(self.layers[l - 1].z)


    def update_parameters(self, learning_rate):
        for key in self.parameters:
            if key.startswith('W'):
                self.parameters[key] -= learning_rate * self.gradients['d' + key]
            elif key.startswith('b'):
                self.parameters[key] -= learning_rate * self.gradients['d' + key]

        # Update layer parameters to match
        for i, layer in enumerate(self.layers):
            layer.weights = self.parameters[f'W{i}']
            layer.biases = self.parameters[f'b{i}']


## Step Seven :Training Loop

---

The training loop orchestrates the complete learning process by repeatedly executing three key steps: forward propagation, backpropagation, and parameter updates. This iterative process continues for a specified number of epochs or until convergence.

### Mini-Batch Gradient Descent

---

We implement mini-batch gradient descent, which strikes a balance between:

- Batch gradient descent (uses all training data, stable but slow)
- Stochastic gradient descent (uses one sample at a time, fast but noisy)

The algorithm:

- Shuffle the training data to ensure random ordering
- Split data into mini-batches of specified size
- For each mini-batch:
- Perform forward propagation to compute predictions
- Calculate loss between predictions and true labels
- Perform backpropagation to compute gradients
- Update parameters using the computed gradients
- Repeat for specified number of epochs

This approach provides a good compromise between computational efficiency and convergence stability.

### Loss Tracking

---

We track the loss after each epoch to monitor training progress:

- Decreasing loss indicates the model is learning
- Plateauing loss may suggest convergence or need for learning rate adjustment
- Increasing loss may indicate too high learning rate or other issues

In [56]:
class NeuralNetworkClassifier:
    def __init__(self, input_size, activation, hidden_layers=[64, 32], output_size=2):
        total_layers = len(hidden_layers) + 1  # hidden layers + output layer
        self.layers = np.empty(total_layers, dtype=object)
        self.input_size = input_size
        self.output_size = output_size
        self.parameters = {}
        self.gradients = {}
        self.loss_history = []

        # Track dimensions for layer connections
        prev_size = input_size

        # Add hidden layers
        for i, layer_size in enumerate(hidden_layers):
            layer = Layer(prev_size, layer_size, activation)
            self.layers[i] = layer
            self.parameters[f'W{i}'] = layer.weights
            self.parameters[f'b{i}'] = layer.biases
            prev_size = layer_size

        # Add output layer (always uses softmax for classification)
        output_layer = Layer(prev_size, output_size, 'softmax')
        self.layers[-1] = output_layer
        self.parameters[f'W{len(hidden_layers)}'] = output_layer.weights
        self.parameters[f'b{len(hidden_layers)}'] = output_layer.biases


    def forward(self, X):
        # Propagate input through all layers.
        input = X
        for layer in self.layers:
            input = layer.forward(input)
        return input


    def compute_loss(self, y_pred, y_true, epsilon=1e-12):
        # Ensure numerical stability by clipping predictions
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)

        # Compute cross-entropy loss for each sample
        # -y_true * log(y_pred) and sum across classes
        sample_losses = -np.sum(y_true * np.log(y_pred), axis=0)

        # Return average loss J across the batch
        J = np.mean(sample_losses)
        return J


    def backward(self, X, y_true, y_pred):
        batch_size = y_true.shape[1]

        # Start with output layer gradient (softmax cross-entropy derivative)
        dZ = (y_pred - y_true) / batch_size  # Average gradient over batch

        # Backpropagate through layers
        for l in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[l]

            # Compute gradients for current layer
            self.gradients[f'dW{l}'] = np.dot(dZ, layer.input.T)
            self.gradients[f'db{l}'] = np.sum(dZ, axis=1, keepdims=True)

            if l > 0:  # Compute gradient for previous layer
                dA_prev = np.dot(layer.weights.T, dZ)
                dZ = dA_prev * self.layers[l - 1].activation_derivative(self.layers[l - 1].z)


    def update_parameters(self, learning_rate):
        for key in self.parameters:
            if key.startswith('W'):
                self.parameters[key] -= learning_rate * self.gradients['d' + key]
            elif key.startswith('b'):
                self.parameters[key] -= learning_rate * self.gradients['d' + key]

        # Update layer parameters to match
        for i, layer in enumerate(self.layers):
            layer.weights = self.parameters[f'W{i}']
            layer.biases = self.parameters[f'b{i}']


    def fit(self, X, y, learning_rate=0.01, epochs=100, batch_size=32, verbose=True):
        n_samples = X.shape[1]

        for epoch in range(epochs):
            epoch_loss = 0
            n_batches = 0

            # Shuffle data
            permutation = np.random.permutation(n_samples)
            X_shuffled = X[:, permutation]
            y_shuffled = y[:, permutation]

            # Process mini-batches
            for i in range(0, n_samples, batch_size):
                # Get mini-batch
                X_batch = X_shuffled[:, i:i + batch_size]
                y_batch = y_shuffled[:, i:i + batch_size]

                # Forward pass
                y_pred = self.forward(X_batch)

                # Compute loss
                loss = self.compute_loss(y_pred, y_batch)
                epoch_loss += loss
                n_batches += 1

                # Backward pass
                self.backward(X_batch, y_batch, y_pred)

                # Update parameters
                self.update_parameters(learning_rate)

            # Record average loss for this epoch
            avg_loss = epoch_loss / n_batches
            self.loss_history.append(avg_loss)

            # Print progress
            if verbose and (epoch % 10 == 0 or epoch == epochs - 1):
                print(f"Epoch {epoch}: Loss = {avg_loss:.4f}")


## Step Eight: Prediction

---

Once trained, the neural network can make predictions on new data. The prediction process involves:

- Class Prediction (predict method): Returns the most likely class for each input sample by selecting the class with the highest probability after forward propagation.
- Probability Prediction (predict_proba method): Returns the raw probability scores for each class, providing a measure of prediction confidence.

These methods enable both categorical predictions and probabilistic outputs, supporting various evaluation scenarios and decision-making processes.


In [57]:
class NeuralNetworkClassifier:
    def __init__(self, input_size, activation, hidden_layers=[64, 32], output_size=2):
        total_layers = len(hidden_layers) + 1  # hidden layers + output layer
        self.layers = np.empty(total_layers, dtype=object)
        self.input_size = input_size
        self.output_size = output_size
        self.parameters = {}
        self.gradients = {}
        self.loss_history = []

        # Track dimensions for layer connections
        prev_size = input_size

        # Add hidden layers
        for i, layer_size in enumerate(hidden_layers):
            layer = Layer(prev_size, layer_size, activation)
            self.layers[i] = layer
            self.parameters[f'W{i}'] = layer.weights
            self.parameters[f'b{i}'] = layer.biases
            prev_size = layer_size

        # Add output layer (always uses softmax for classification)
        output_layer = Layer(prev_size, output_size, 'softmax')
        self.layers[-1] = output_layer
        self.parameters[f'W{len(hidden_layers)}'] = output_layer.weights
        self.parameters[f'b{len(hidden_layers)}'] = output_layer.biases


    def forward(self, X):
        # Propagate input through all layers.
        input = X
        for layer in self.layers:
            input = layer.forward(input)
        return input


    def compute_loss(self, y_pred, y_true, epsilon=1e-12):
        # Ensure numerical stability by clipping predictions
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)

        # Compute cross-entropy loss for each sample
        # -y_true * log(y_pred) and sum across classes
        sample_losses = -np.sum(y_true * np.log(y_pred), axis=0)

        # Return average loss J across the batch
        J = np.mean(sample_losses)
        return J


    def backward(self, X, y_true, y_pred):
        batch_size = y_true.shape[1]

        # Start with output layer gradient (softmax cross-entropy derivative)
        dZ = (y_pred - y_true) / batch_size  # Average gradient over batch

        # Backpropagate through layers
        for l in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[l]

            # Compute gradients for current layer
            self.gradients[f'dW{l}'] = np.dot(dZ, layer.input.T)
            self.gradients[f'db{l}'] = np.sum(dZ, axis=1, keepdims=True)

            if l > 0:  # Compute gradient for previous layer
                dA_prev = np.dot(layer.weights.T, dZ)
                dZ = dA_prev * self.layers[l - 1].activation_derivative(self.layers[l - 1].z)


    def update_parameters(self, learning_rate):
        for key in self.parameters:
            if key.startswith('W'):
                self.parameters[key] -= learning_rate * self.gradients['d' + key]
            elif key.startswith('b'):
                self.parameters[key] -= learning_rate * self.gradients['d' + key]

        # Update layer parameters to match
        for i, layer in enumerate(self.layers):
            layer.weights = self.parameters[f'W{i}']
            layer.biases = self.parameters[f'b{i}']


    def fit(self, X, y, learning_rate=0.01, epochs=100, batch_size=32, verbose=True):
        n_samples = X.shape[1]

        for epoch in range(epochs):
            epoch_loss = 0
            n_batches = 0

            # Shuffle data
            permutation = np.random.permutation(n_samples)
            X_shuffled = X[:, permutation]
            y_shuffled = y[:, permutation]

            # Process mini-batches
            for i in range(0, n_samples, batch_size):
                # Get mini-batch
                X_batch = X_shuffled[:, i:i + batch_size]
                y_batch = y_shuffled[:, i:i + batch_size]

                # Forward pass
                y_pred = self.forward(X_batch)

                # Compute loss
                loss = self.compute_loss(y_pred, y_batch)
                epoch_loss += loss
                n_batches += 1

                # Backward pass
                self.backward(X_batch, y_batch, y_pred)

                # Update parameters
                self.update_parameters(learning_rate)

            # Record average loss for this epoch
            avg_loss = epoch_loss / n_batches
            self.loss_history.append(avg_loss)

            # Print progress
            if verbose and (epoch % 10 == 0 or epoch == epochs - 1):
                print(f"Epoch {epoch}: Loss = {avg_loss:.4f}")


    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=0)


    def predict_proba(self, X):
        return self.forward(X)

## Step Nine: Evaluation

---

Model evaluation is essential to assess performance on unseen data. The evaluation metrics include:

- Accuracy: Proportion of correctly classified samples
- Precision: Ratio of true positives to all predicted positives for each class
- Recall: Ratio of true positives to all actual positives for each class
- F1-Score: Harmonic mean of precision and recall
- Macro Averages: Mean of per-class metrics, providing class-balanced performance measures

These metrics offer a comprehensive view of model performance across all classes, particularly important for imbalanced datasets.

In [58]:
class NeuralNetworkClassifier:
    def __init__(self, input_size, activation, hidden_layers=[64, 32], output_size=2):
        total_layers = len(hidden_layers) + 1  # hidden layers + output layer
        self.layers = np.empty(total_layers, dtype=object)
        self.input_size = input_size
        self.output_size = output_size
        self.parameters = {}
        self.gradients = {}
        self.loss_history = []

        # Track dimensions for layer connections
        prev_size = input_size

        # Add hidden layers
        for i, layer_size in enumerate(hidden_layers):
            layer = Layer(prev_size, layer_size, activation)
            self.layers[i] = layer
            self.parameters[f'W{i}'] = layer.weights
            self.parameters[f'b{i}'] = layer.biases
            prev_size = layer_size

        # Add output layer (always uses softmax for classification)
        output_layer = Layer(prev_size, output_size, 'softmax')
        self.layers[-1] = output_layer
        self.parameters[f'W{len(hidden_layers)}'] = output_layer.weights
        self.parameters[f'b{len(hidden_layers)}'] = output_layer.biases


    def forward(self, X):
        # Propagate input through all layers.
        input = X
        for layer in self.layers:
            input = layer.forward(input)
        return input


    def compute_loss(self, y_pred, y_true, epsilon=1e-12):
        # Ensure numerical stability by clipping predictions
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)

        # Compute cross-entropy loss for each sample
        # -y_true * log(y_pred) and sum across classes
        sample_losses = -np.sum(y_true * np.log(y_pred), axis=0)

        # Return average loss J across the batch
        J = np.mean(sample_losses)
        return J


    def backward(self, X, y_true, y_pred):
        batch_size = y_true.shape[1]

        # Start with output layer gradient (softmax cross-entropy derivative)
        dZ = (y_pred - y_true) / batch_size  # Average gradient over batch

        # Backpropagate through layers
        for l in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[l]

            # Compute gradients for current layer
            self.gradients[f'dW{l}'] = np.dot(dZ, layer.input.T)
            self.gradients[f'db{l}'] = np.sum(dZ, axis=1, keepdims=True)

            if l > 0:  # Compute gradient for previous layer
                dA_prev = np.dot(layer.weights.T, dZ)
                dZ = dA_prev * self.layers[l - 1].activation_derivative(self.layers[l - 1].z)


    def update_parameters(self, learning_rate):
        for key in self.parameters:
            if key.startswith('W'):
                self.parameters[key] -= learning_rate * self.gradients['d' + key]
            elif key.startswith('b'):
                self.parameters[key] -= learning_rate * self.gradients['d' + key]

        # Update layer parameters to match
        for i, layer in enumerate(self.layers):
            layer.weights = self.parameters[f'W{i}']
            layer.biases = self.parameters[f'b{i}']


    def fit(self, X, y, learning_rate=0.01, epochs=100, batch_size=32, verbose=True):
        n_samples = X.shape[1]

        for epoch in range(epochs):
            epoch_loss = 0
            n_batches = 0

            # Shuffle data
            permutation = np.random.permutation(n_samples)
            X_shuffled = X[:, permutation]
            y_shuffled = y[:, permutation]

            # Process mini-batches
            for i in range(0, n_samples, batch_size):
                # Get mini-batch
                X_batch = X_shuffled[:, i:i + batch_size]
                y_batch = y_shuffled[:, i:i + batch_size]

                # Forward pass
                y_pred = self.forward(X_batch)

                # Compute loss
                loss = self.compute_loss(y_pred, y_batch)
                epoch_loss += loss
                n_batches += 1

                # Backward pass
                self.backward(X_batch, y_batch, y_pred)

                # Update parameters
                self.update_parameters(learning_rate)

            # Record average loss for this epoch
            avg_loss = epoch_loss / n_batches
            self.loss_history.append(avg_loss)

            # Print progress
            if verbose and (epoch % 10 == 0 or epoch == epochs - 1):
                print(f"Epoch {epoch}: Loss = {avg_loss:.4f}")


    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=0)


    def predict_proba(self, X):
        return self.forward(X)


    def evaluate(self, X, y):
        # Get predictions
        y_pred = self.predict(X)
        y_true = np.argmax(y, axis=0)

        # Calculate accuracy
        accuracy = np.mean(y_pred == y_true)

        # Calculate precision, recall, and F1 for each class
        precision = []
        recall = []
        f1 = []

        for class_idx in range(self.output_size):
            # True positives, false positives, false negatives
            tp = np.sum((y_pred == class_idx) & (y_true == class_idx))
            fp = np.sum((y_pred == class_idx) & (y_true != class_idx))
            fn = np.sum((y_pred != class_idx) & (y_true == class_idx))

            # Avoid division by zero
            prec = tp / (tp + fp) if (tp + fp) > 0 else 0
            rec = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1_score = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0

            precision.append(prec)
            recall.append(rec)
            f1.append(f1_score)

        # Calculate macro averages
        macro_precision = np.mean(precision)
        macro_recall = np.mean(recall)
        macro_f1 = np.mean(f1)

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'macro_precision': macro_precision,
            'macro_recall': macro_recall,
            'macro_f1': macro_f1
        }

## Step Ten: Testing

---

### Dataset overview

---

We tested our neural network implementation on the Digits dataset from scikit-learn:
- Samples: 1,797 handwritten digit images
- Features: 64 (8×8 pixel images flattened to 1D vectors)
- Classes: 10 (digits 0-9)
- Task: Multi-class classification of handwritten digits

### Preprocessing Steps

---

1. Feature Standardization: Pixel values were standardized to have zero mean and unit variance
2. Label Encoding: Class labels were one-hot encoded for compatibility with our neural network
3. Train-Test Split: Data was divided into 80% training and 20% testing sets

### Model Architecture

---

The neural network was configured with:

- Input Layer: 64 neurons (matching the flattened image dimensions)
- Hidden Layers: Two layers with 64 and 32 neurons respectively
- Output Layer: 10 neurons (one for each digit class)
- Activation Function: ReLU for hidden layers, Softmax for output layer
- Learning Rate: 0.01
- Batch Size: 32
- Epochs: 500


In [59]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [60]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

# Load the Digits dataset
digits = load_digits()
X = digits.data  # Features: 64 (flattened image pixels)
y = digits.target  # Labels: 10 (0-9)

# Preprocess the data
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=3
)

# Transpose to match our model's expected input format (features x samples)
X_train = X_train.T
X_test = X_test.T
y_train = y_train.T
y_test = y_test.T

model = NeuralNetworkClassifier(
    input_size=64,  # 64 features (pixels)
    activation='relu',  # ReLU activation for hidden layers
    hidden_layers=[64, 32],  # Larger hidden layers for this more complex problem
    output_size=10  # 10 classes (digits 0-9)
)

# Train the model
print("\nTraining the neural network on Digits dataset...\n")
model.fit(X_train, y_train, learning_rate=0.01, epochs=500, batch_size=32, verbose=True)

# Evaluate the model
results = model.evaluate(X_test, y_test)
print(f"\nModel Performance on Test Set:")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {[f'{p:.4f}' for p in results['precision']]}")
print(f"Recall: {[f'{r:.4f}' for r in results['recall']]}")
print(f"F1-Score: {[f'{f:.4f}' for f in results['f1']]}")

# Make predictions on the test set
predictions = model.predict(X_test)
true_labels = np.argmax(y_test, axis=0)

print(f"\nSample predictions vs true labels:")
for i in range(min(15, len(predictions))):
    print(f"Predicted: {predictions[i]}, True: {true_labels[i]}")


Training the neural network on Digits dataset...

Epoch 0: Loss = 2.3426
Epoch 10: Loss = 0.5462
Epoch 20: Loss = 0.2288
Epoch 30: Loss = 0.1382
Epoch 40: Loss = 0.0964
Epoch 50: Loss = 0.0716
Epoch 60: Loss = 0.0554
Epoch 70: Loss = 0.0441
Epoch 80: Loss = 0.0359
Epoch 90: Loss = 0.0297
Epoch 100: Loss = 0.0250
Epoch 110: Loss = 0.0213
Epoch 120: Loss = 0.0184
Epoch 130: Loss = 0.0160
Epoch 140: Loss = 0.0141
Epoch 150: Loss = 0.0125
Epoch 160: Loss = 0.0112
Epoch 170: Loss = 0.0101
Epoch 180: Loss = 0.0092
Epoch 190: Loss = 0.0084
Epoch 200: Loss = 0.0078
Epoch 210: Loss = 0.0072
Epoch 220: Loss = 0.0067
Epoch 230: Loss = 0.0062
Epoch 240: Loss = 0.0058
Epoch 250: Loss = 0.0055
Epoch 260: Loss = 0.0051
Epoch 270: Loss = 0.0049
Epoch 280: Loss = 0.0046
Epoch 290: Loss = 0.0044
Epoch 300: Loss = 0.0041
Epoch 310: Loss = 0.0039
Epoch 320: Loss = 0.0038
Epoch 330: Loss = 0.0036
Epoch 340: Loss = 0.0034
Epoch 350: Loss = 0.0033
Epoch 360: Loss = 0.0032
Epoch 370: Loss = 0.0030
Epoch 380:

## Summary

---

This notebook has successfully demonstrated the implementation of a neural network for classification from scratch using only NumPy. The project provides a comprehensive foundation in deep learning fundamentals, covering all critical aspects of neural network design, training, and evaluation.

### Key Achievements

---

1. Complete Implementation: Built a functional neural network with configurable architecture, proper weight initialization, forward/backward propagation, and training loop
2. Mathematical Foundation: Incorporated proper mathematical formulations for activation functions, loss calculation, and gradient computation
3. Practical Application: Successfully tested the implementation on the Digits dataset with strong performance
4. Evaluation Metrics: Implemented comprehensive evaluation including accuracy, precision, recall, and F1-score

### Performance Summary

---

The neural network achieved competitive performance on the challenging Digits dataset, demonstrating its capability to handle real-world classification problems with multiple classes and features.

### Next Steps and Future Improvements

---

While this implementation provides a solid foundation, several enhancements could further improve its robustness and performance:

1. Regularization Techniques

- L2 Regularization: Add weight decay to prevent overfitting
- Dropout: Implement random neuron deactivation during training
- Early Stopping: Monitor validation loss to prevent overtraining

2. Advanced Optimization

- Adam Optimizer: Implement adaptive learning rate optimization
- Learning Rate Scheduling: Add decay or cosine annealing for better convergence
- Momentum: Incorporate momentum in gradient updates for smoother optimization

3. Model Validation

- K-Fold Cross Validation: Implement proper cross-validation for more reliable performance estimates
- Validation Set: Separate training data into training/validation splits for hyperparameter tuning
- Hyperparameter Optimization: Add grid search or random search for optimal parameter selection

4. Architectural Enhancements

- Batch Normalization: Implement normalization between layers for faster training
- Different Architectures: Support for convolutional layers (CNNs) for image data
- Residual Connections: Add skip connections for deeper networks

5. Visualization and Analysis

- Training Curves: Plot loss and accuracy over epochs
- Confusion Matrices: Visualize classification performance
- Feature Visualization: Explore what the network has learned

6. Additional Datasets

- Iris Dataset: Test the model on this classic classification benchmark
- More Complex Datasets: Evaluate performance on larger datasets like CIFAR-10 or MNIST
